In [ ]:
import json
import math

import pandas as pd
import numpy as np
import folium

In [ ]:
fp7projects = pd.read_csv('data/research_projects_2007_2013/cordis-fp7projects.csv', sep=';', decimal=',')
fp7projects = fp7projects[fp7projects.coordinatorCountry.str.contains('UK') | fp7projects.participantCountries.str.contains('UK')]
fp7projects.shape

In [ ]:
fp7projects[fp7projects.coordinatorCountry.str.contains('UK')]['coordinator'].nunique()

In [ ]:
 fp7projects.columns

In [ ]:
fp7projects.head()

In [ ]:
# I see some trailing spaces and lower case codes.
fp7projects['coordinatorclean'] = fp7projects['coordinator'].str.replace("THE", "").str.replace("LIMITED", "").str.replace("PLC", "").str.replace("LLC", "").str.replace("LBG", "").str.replace("LTD", "").str.replace("HIGHER EDUCATION CORPORATION", "").str.replace("UNIVERSITY COURT OF", "").str.replace("CHANCELLOR MASTERS AND SCHOLARS OF", "").str.replace("CHANCELLOR, MASTERS AND SCHOLARS OF", "").str.strip().str.upper()

In [ ]:
fp7projectscoord = fp7projects[fp7projects.coordinatorCountry.str.contains('UK')]
fp7projectscoord.shape

In [ ]:
grid = pd.read_csv('data/grid20180625/grid.csv')
grid['nameclean'] = grid['Name'].str.replace("THE", "").str.replace("LIMITED", "").str.replace("PLC", "").str.replace("LLC", "").str.replace("LBG", "").str.replace("LTD", "").str.replace("HIGHER EDUCATION CORPORATION", "").str.replace("UNIVERSITY COURT OF", "").str.replace("CHANCELLOR MASTERS AND SCHOLARS OF", "").str.replace("CHANCELLOR, MASTERS AND SCHOLARS OF", "").str.strip().str.upper()
grid.head()

In [ ]:
grid_addresses = pd.read_csv('data/grid20180625/full_tables/addresses.csv', low_memory=False)
grid_addresses.head()

In [ ]:
fp7projectscoord = pd.merge(fp7projectscoord, grid, how='left', left_on='coordinatorclean', right_on='nameclean')
fp7projectscoord.shape

In [ ]:
fp7projectscoord = pd.merge(fp7projectscoord, grid_addresses, how='left', left_on='ID', right_on='grid_id')
fp7projectscoord.head()

In [ ]:

unmatched = fp7projectscoord[pd.isnull(fp7projectscoord.lat)]['coordinatorclean']

unmatched = unmatched.to_frame()
unmatched.groupby('coordinatorclean').nunique()[60:80]

In [ ]:
fp7projectscoord.columns

In [ ]:
for institute in fp7projectscoord.itertuples():
    if not (pd.isnull(institute[33])):
        print([ institute[33], institute[34] ])

In [ ]:
mapit = folium.Map(location=[51.50810, -0.14077], zoom_start=10 )
for institute in fp7projectscoord.itertuples():
    if not pd.isnull(institute[33]):
        folium.Marker(location=[ institute[33], institute[34] ]).add_to(mapit)
mapit

In [ ]:
def make_coordis_data_geo_json(coordis_data):
    def make_feature(row):
        properties = {
            property: row[property]
            for property in ['acronym', 'title', 'startDate', 'endDate',
                             'projectUrl', 'totalCost', 'ecMaxContribution']
            if str(row[property]) != 'nan'
        }
        return {
            'type': 'Feature',
            'geometry': {
                "type": "Point",
                "coordinates": [row['lng'], row['lat']]
            },
            'properties': properties
        }
    features = list(coordis_data.apply(make_feature, axis=1))
    return { 'type': 'FeatureCollection', 'features': features }
with open('data/coordis_data.geo.json', 'w') as file:
    json.dump(make_coordis_data_geo_json(
        fp7projectscoord[~pd.isnull(fp7projectscoord['lat'])]
    ), file, sort_keys=True)